# Table of Contents
 <p>

In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
import GPy
import numpy as np

nobsv=3000
X = np.random.normal(size=(nobsv,2))
Y = np.random.normal(size=(nobsv,1))
se = GPy.kern.RBF(2, lengthscale=1.0)

gp_se = GPy.models.GPRegression(X, Y, se)

In [3]:
import gc
gc.collect()
%timeit -n2 gp_se.parameters_changed()

2 loops, best of 3: 1.34 s per loop


In [4]:
rq = GPy.kern.RatQuad(2, lengthscale=1.0, power=1.0)
gp_rq = GPy.models.GPRegression(X, Y, rq)

In [5]:
%timeit gp_rq.parameters_changed()

1 loop, best of 3: 1.76 s per loop


In [6]:
gp_sum = GPy.models.GPRegression(X, Y, se+rq)

In [7]:
%timeit gp_sum.parameters_changed()

1 loop, best of 3: 1.85 s per loop


In [8]:
gp_prod = GPy.models.GPRegression(X, Y, se*rq)

In [9]:
%timeit gp_prod.parameters_changed()

1 loop, best of 3: 1.95 s per loop


In [10]:
se2 = GPy.kern.RBF(2, lengthscale=1.0)

In [11]:
gp_sum = GPy.models.GPRegression(X, Y, se+se2+rq)

In [12]:
%timeit gp_sum.parameters_changed()

1 loop, best of 3: 2.1 s per loop


In [13]:
gp_prod = GPy.models.GPRegression(X, Y, (se+se2)*rq)

In [14]:
%timeit gp_prod.parameters_changed()

1 loop, best of 3: 2.19 s per loop


In [15]:
semask = GPy.kern.RBF(1, lengthscale=1.0, active_dims=[0])
rqmask = GPy.kern.RatQuad(1, lengthscale=1.0, power=1.0, active_dims=[1])

In [16]:
gp = GPy.models.GPRegression(X, Y, semask)
%timeit gp.parameters_changed()

 /Users/imolk/bin/venv_python3/lib/python3.5/site-packages/GPy/core/gp.py:86: UserWarning:Your kernel has a different input dimension 1 then the given X dimension 2. Be very sure this is what you want and you have not forgotten to set the right input dimenion in your kernel


1 loop, best of 3: 1.24 s per loop


In [17]:
gp = GPy.models.GPRegression(X, Y, semask+rqmask)
%timeit gp.parameters_changed()

1 loop, best of 3: 1.94 s per loop


In [18]:
sefix = GPy.kern.RBF(2, lengthscale=1.0)
sefix.variance.fix()
gp = GPy.models.GPRegression(X, Y, sefix)
%timeit gp.parameters_changed()

1 loop, best of 3: 1.29 s per loop


In [19]:
%time gp.parameters_changed()

CPU times: user 2.46 s, sys: 168 ms, total: 2.63 s
Wall time: 1.31 s
